# Explore here

In [74]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
#Detección  automáticamente de columnas que son números y cuáles son texto
from sklearn.compose import make_column_selector
# Por asi decirlo el embudo donde se mete todo para que luego se procese
from sklearn.pipeline import Pipeline
# Modelo 
from sklearn.linear_model import LogisticRegression

# 1. Puesta en marcha
   - Cargamos el archivo
   - Limpiamos los "?" y valores vacíos
   - Creamos la columna de 'éxito' (1 si gana >50K, 0 si no)


In [75]:
df = pd.read_csv("https://breathecode.herokuapp.com/asset/internal-link?id=2326&path=adult-census-income.csv")
df.head(10)

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K
5,34,Private,216864,HS-grad,9,Divorced,Other-service,Unmarried,White,Female,0,3770,45,United-States,<=50K
6,38,Private,150601,10th,6,Separated,Adm-clerical,Unmarried,White,Male,0,3770,40,United-States,<=50K
7,74,State-gov,88638,Doctorate,16,Never-married,Prof-specialty,Other-relative,White,Female,0,3683,20,United-States,>50K
8,68,Federal-gov,422013,HS-grad,9,Divorced,Prof-specialty,Not-in-family,White,Female,0,3683,40,United-States,<=50K
9,41,Private,70037,Some-college,10,Never-married,Craft-repair,Unmarried,White,Male,0,3004,60,?,>50K


In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education.num   32561 non-null  int64 
 5   marital.status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital.gain    32561 non-null  int64 
 11  capital.loss    32561 non-null  int64 
 12  hours.per.week  32561 non-null  int64 
 13  native.country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [77]:
df.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education.num     0
marital.status    0
occupation        0
relationship      0
race              0
sex               0
capital.gain      0
capital.loss      0
hours.per.week    0
native.country    0
income            0
dtype: int64

In [78]:
df.replace("?", pd.NA, inplace=True)
#aqui actua como un buscador y borra todo lo que este vacio
df.dropna(inplace=True)

In [79]:
df['taza_exito'] = df['income'].apply(lambda x: 1 if '>50K' in x else 0)
df["taza_exito"].head(10)

1     0
3     0
4     0
5     0
6     0
7     1
8     0
10    1
11    1
12    1
Name: taza_exito, dtype: int64

- Quiero ver quienes son los 10 que ganan mas de 50.

In [80]:
billetuos = df[df['taza_exito'] == 1]
billetuos.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income,taza_exito
7,74,State-gov,88638,Doctorate,16,Never-married,Prof-specialty,Other-relative,White,Female,0,3683,20,United-States,>50K,1
10,45,Private,172274,Doctorate,16,Divorced,Prof-specialty,Unmarried,Black,Female,0,3004,35,United-States,>50K,1
11,38,Self-emp-not-inc,164526,Prof-school,15,Never-married,Prof-specialty,Not-in-family,White,Male,0,2824,45,United-States,>50K,1
12,52,Private,129177,Bachelors,13,Widowed,Other-service,Not-in-family,White,Female,0,2824,20,United-States,>50K,1
13,32,Private,136204,Masters,14,Separated,Exec-managerial,Not-in-family,White,Male,0,2824,55,United-States,>50K,1


# 2 . Limpieza beaby

- He encontrado esta forma de hacer la limpiza sin la necesidad de separar las columnas en categóricas o numericas ; vamos a ver que tal funciona:  

In [81]:


limpiador = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), make_column_selector(dtype_include=['int64', 'float64'])),
        ("cat", OneHotEncoder(handle_unknown='ignore'), make_column_selector(dtype_include=['object']))
    ])

# 3 . Separación de las variables.

In [82]:
variables_predictoras = ['age', 'education', 'marital.status', 'occupation', 'hours.per.week', 'sex', 'native.country']

X = df[variables_predictoras]

y = df['taza_exito']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2.1 - Comprobar que el paso 2 funciono

In [83]:
limpios = limpiador.fit_transform(X_train)
print(limpios[0])

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 7 stored elements and shape (1, 82)>
  Coords	Values
  (0, 0)	-0.260418894797645
  (0, 1)	0.33973131522373556
  (0, 17)	1.0
  (0, 20)	1.0
  (0, 36)	1.0
  (0, 40)	1.0
  (0, 79)	1.0


In [84]:
type(limpios)

scipy.sparse._csr.csr_matrix

In [85]:
colummnas_creo_Onehot = limpiador.get_feature_names_out()
colummnas_creo_Onehot

array(['num__age', 'num__hours.per.week', 'cat__education_10th',
       'cat__education_11th', 'cat__education_12th',
       'cat__education_1st-4th', 'cat__education_5th-6th',
       'cat__education_7th-8th', 'cat__education_9th',
       'cat__education_Assoc-acdm', 'cat__education_Assoc-voc',
       'cat__education_Bachelors', 'cat__education_Doctorate',
       'cat__education_HS-grad', 'cat__education_Masters',
       'cat__education_Preschool', 'cat__education_Prof-school',
       'cat__education_Some-college', 'cat__marital.status_Divorced',
       'cat__marital.status_Married-AF-spouse',
       'cat__marital.status_Married-civ-spouse',
       'cat__marital.status_Married-spouse-absent',
       'cat__marital.status_Never-married',
       'cat__marital.status_Separated', 'cat__marital.status_Widowed',
       'cat__occupation_Adm-clerical', 'cat__occupation_Armed-Forces',
       'cat__occupation_Craft-repair', 'cat__occupation_Exec-managerial',
       'cat__occupation_Farming-fishin

# 4 - Creación y entreno del modelo 

In [86]:

embudo_de_procesamiento = Pipeline(steps=[
    ('preprocessor', limpiador), 
    ('classifier', LogisticRegression(max_iter=1000)) 
])

embudo_de_procesamiento.fit(X_train, y_train)

,steps,"[('preprocessor', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


# 5 Prueba de que el modelo funciona.

In [87]:

nota_final = embudo_de_procesamiento.score(X_test, y_test)

# 2. Mostramos el resultado de una forma bonita
print(f"¡Mi robot sacó un: {nota_final * 100:.2f} /de  100")

¡Mi robot sacó un: 82.10 /de  100


# Plantear cómo vas a estructurar tu sistema de recomendación:

1. ¿Qué se quiere recomendar?
Queremos recomendar un "Salto de Nivel". Imagina que el sistema es como un entrenador personal de tu carrera. Su trabajo no es solo decirte si ahora mismo vas bien o mal, sino responderte a la pregunta: "¿Qué es lo mínimo que tengo que cambiar para entrar en el club de los que ganan más de 50k?".

2. ¿Cuál será el "usuario"?
Lo vamos a llamar el "Candidato Estancado". Es alguien que ya se esfuerza, pero siente que no avanza. Por ejemplo: una persona de 35 años que trabaja un montón de horas, pero que quizás está en el lugar equivocado o le falta un "papel" para que le paguen más.

3. ¿Qué variables definen su perfil?

-  hours.per.week
- education
- marital.status
- occupation
-hours.per.week

4. ¿Cómo es el sistema? 
Enfoque de "Búsqueda por Fuerza Bruta"

¿Cómo funciona? En lugar de que tú le digas al robot qué probar, el robot prueba TODO.

El robot agarra a tu usuario y, en un segundo, lo "disfraza" de médico, luego de ingeniero, luego de jefe... y así con todos los trabajos que conoce.

Al final, el robot te hace una lista de cuáles de esos "disfraces" son los que de verdad funcionan para ganar más dinero.

In [95]:

usuario_base = perfil_usuario.copy()
usuario_base['age'] = [35]
usuario_base['education'] = ['Masters']
usuario_base['marital.status'] = ['Married-civ-spouse'] # <-- ¡Este es el truco!
usuario_base['hours.per.week'] = [50]


todos_los_trabajos = X_train['occupation'].unique()

print("Buscando el mejor trabajo para ti...")


for trabajo in todos_los_trabajos:
    usuario_base['occupation'] = trabajo
    puedo_ganar_mas = embudo_de_procesamiento.predict(usuario_base)[0]
    
    if puedo_ganar_mas == 1:
        print(f"🌟 RECOMENDACIÓN: Si te cambias a '{trabajo}', podrías ganar +50k.")
    else:
        
        print(f"   - Revisando '{trabajo}': No es suficiente.")    

Buscando el mejor trabajo para ti...
🌟 RECOMENDACIÓN: Si te cambias a 'Sales', podrías ganar +50k.
🌟 RECOMENDACIÓN: Si te cambias a 'Adm-clerical', podrías ganar +50k.
🌟 RECOMENDACIÓN: Si te cambias a 'Exec-managerial', podrías ganar +50k.
🌟 RECOMENDACIÓN: Si te cambias a 'Handlers-cleaners', podrías ganar +50k.
🌟 RECOMENDACIÓN: Si te cambias a 'Craft-repair', podrías ganar +50k.
🌟 RECOMENDACIÓN: Si te cambias a 'Prof-specialty', podrías ganar +50k.
   - Revisando 'Other-service': No es suficiente.
🌟 RECOMENDACIÓN: Si te cambias a 'Transport-moving', podrías ganar +50k.
🌟 RECOMENDACIÓN: Si te cambias a 'Machine-op-inspct', podrías ganar +50k.
🌟 RECOMENDACIÓN: Si te cambias a 'Tech-support', podrías ganar +50k.
   - Revisando 'Farming-fishing': No es suficiente.
🌟 RECOMENDACIÓN: Si te cambias a 'Protective-serv', podrías ganar +50k.
   - Revisando 'Priv-house-serv': No es suficiente.
🌟 RECOMENDACIÓN: Si te cambias a 'Armed-Forces', podrías ganar +50k.
